# Restaurant Recommender AI

### Objective:
Building a Restaurant Recommender AI system to assist users in finding the perfect dining spot based on their preferences. The application utilizes OpenAI model to generate restaurant recommendations by considering various factors such as location, cuisine preferences, restaurant type and user ratings.

In [5]:
# Import the libraries
import pandas as pd
from IPython.display import display, HTML
# Set the display width to control the output width
pd.set_option('display.width', 100)

In [6]:
# Read the Restaurants dataset 
rests_df = pd.read_csv('blr_all_restaurants.csv')
rests_df.shape

(6984, 10)

In [7]:
rests_df.head()

,restaurant name,restaurant type,rate (out of 5),num of ratings,avg cost (two people),online_order,table booking,cuisines type,area,local address
0,#FeelTheROLL,Quick Bites,3.4,7,200.0,No,No,Fast Food,Bellandur,Bellandur
1,#L-81 Cafe,Quick Bites,3.9,48,400.0,Yes,No,"Fast Food, Beverages","Byresandra,Tavarekere,Madiwala",HSR
2,#refuel,Cafe,3.7,37,400.0,Yes,No,"Cafe, Beverages",Bannerghatta Road,Bannerghatta Road
3,'@ Biryani Central,Casual Dining,2.7,135,550.0,Yes,No,"Biryani, Mughlai, Chinese",Marathahalli,Marathahalli
4,'@ The Bbq,Casual Dining,2.8,40,700.0,Yes,No,"BBQ, Continental, North Indian, Chinese, Bever...",Bellandur,Bellandur


In [8]:
rests_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6984 entries, 0 to 6983
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   restaurant name        6984 non-null   object 
 1   restaurant type        6984 non-null   object 
 2   rate (out of 5)        6984 non-null   float64
 3   num of ratings         6984 non-null   int64  
 4   avg cost (two people)  6984 non-null   float64
 5   online_order           6984 non-null   object 
 6   table booking          6984 non-null   object 
 7   cuisines type          6984 non-null   object 
 8   area                   6984 non-null   object 
 9   local address          6984 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 545.8+ KB


In [9]:
# Import the libraries
import os, json, ast
import openai

In [10]:
# Read the OpenAI API key
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [11]:
# Define the OpenAI Model used for this project
model='gpt-4o-mini'

![System_Design](system_design.png)

In [10]:
#Initializes the conversation with the system message.
def initialize_conversation():

    system_message = f"""
    You are an intelligent and helpful restaurant recommendation system. 
    Your goal is to understand a user's restaurant preferences and create a detailed user profile dictionary to provide personalized recommendations.

    **Instructions:**

    1.  Engage the user in a natural and friendly conversation to gather information about their restaurant preferences.  Be polite and helpful.
    2.  Focus on extracting details related to the following key aspects of their ideal restaurant:
        *   **Restaurant Type:** (e.g., Casual Dining, Fine Dining, Pub etc.)
        *   **Cuisine Type:** (e.g., North Indian, Thai, Biryani etc.)
        *   **Cuisine Type:** Location (e.g., neighborhood, area etc.).
        *   **Cost for Two:** (price for a meal for two people, pick only the value)
    3.  Use chain-of-thought reasoning to guide the conversation.  This means thinking step-by-step about what information you need and asking questions that will help you obtain it.  Don't just ask all the questions at once; let the conversation flow naturally.
    4.  Continue asking questions until you are confident you have accurately filled all the values in the user profile dictionary.  Don't be afraid to ask clarifying questions.
    5.  Follow the conversation flow outlined below to ensure you achieve the desired outcome.
    6.  Once you have gathered enough information, present the user profile dictionary to the user for confirmation.

    **Conversation Flow (Chain of Thought):**

    *   **Thought 1: Initial Understanding and Confident Values:** Begin by asking broad questions to understand the user's general preferences.  If the user provides information that clearly defines a value for one of the key aspects (Restaurant Type, Cuisine Type, Cost for Two), immediately update the user profile dictionary with that value.
    *   **Thought 2: Filling Remaining Values:** Based on the information already gathered, ask more specific questions to fill in the remaining values in the user profile dictionary.  Use inferences and connections to guide your questions. For example, if the user mentions they like Italian food, you might ask if they prefer a casual pizzeria or a more upscale Italian restaurant.
    *   **Thought 3: Confirmation and Verification:** Once you believe you have gathered enough information, summarize the user's preferences and present the completed user profile dictionary to the user. Ask them to confirm that the information is accurate and if they would like to make any changes.

    **Example User Profile Dictionary (This is just a template, do not use these values unless the user provides them):**

    ```json
    {{
      "Restaurant Type": null,
      "Cuisine Type": null,
      "Location": null,
      "Cost for Two": null
    }}
    """

    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [11]:
# Let's initialise conversation
system_message = initialize_conversation()
print(system_message[0]["content"])


    You are an intelligent and helpful restaurant recommendation system. 
    Your goal is to understand a user's restaurant preferences and create a detailed user profile dictionary to provide personalized recommendations.

    **Instructions:**

    1.  Engage the user in a natural and friendly conversation to gather information about their restaurant preferences.  Be polite and helpful.
    2.  Focus on extracting details related to the following key aspects of their ideal restaurant:
        *   **Restaurant Type:** (e.g., Casual Dining, Fine Dining, Pub etc.)
        *   **Cuisine Type:** (e.g., North Indian, Thai, Biryani etc.)
        *   **Cuisine Type:** Location (e.g., neighborhood, area etc.).
        *   **Cost for Two:** (price for a meal for two people, pick only the value)
    3.  Use chain-of-thought reasoning to guide the conversation.  This means thinking step-by-step about what information you need and asking questions that will help you obtain it.  Don't just ask a

In [12]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are an intelligent and helpful restaurant recommendation system. \n    Your goal is to understand a user\'s restaurant preferences and create a detailed user profile dictionary to provide personalized recommendations.\n\n    **Instructions:**\n\n    1.  Engage the user in a natural and friendly conversation to gather information about their restaurant preferences.  Be polite and helpful.\n    2.  Focus on extracting details related to the following key aspects of their ideal restaurant:\n        *   **Restaurant Type:** (e.g., Casual Dining, Fine Dining, Pub etc.)\n        *   **Cuisine Type:** (e.g., North Indian, Thai, Biryani etc.)\n        *   **Cuisine Type:** Location (e.g., neighborhood, area etc.).\n        *   **Cost for Two:** (price for a meal for two people, pick only the value)\n    3.  Use chain-of-thought reasoning to guide the conversation.  This means thinking step-by-step about what information you need and asking questions th

In [13]:
# Define a Chat Completions API call
def get_chat_completions(input, json_format = False):
    if json_format == True:
        input[0]['content'] += '. Return output in JSON format.'
        chat_response = openai.chat.completions.create(
            model = model, 
            messages = input,
            response_format = { "type": "json_object"},
            seed = 123)
        output = json.loads(chat_response.choices[0].message.content)
    else:
        chat_response = openai.chat.completions.create(
            model = model, 
            messages = input, 
            seed = 123)
        output = chat_response.choices[0].message.content
        
    return output

In [27]:
debug_user_input = "Hi, I am looking for a Fine Dining restaurant, for North Indian food, in Whitefield. Budget around 2000"
debug_conversation.append({"role": "user", "content": debug_user_input})
# Let's look at the debug_conversation list
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are an intelligent and helpful restaurant recommendation system. \n    Your goal is to understand a user\'s restaurant preferences and create a detailed user profile dictionary to provide personalized recommendations.\n\n    **Instructions:**\n\n    1.  Engage the user in a natural and friendly conversation to gather information about their restaurant preferences.  Be polite and helpful.\n    2.  Focus on extracting details related to the following key aspects of their ideal restaurant:\n        *   **Restaurant Type:** (e.g., Casual Dining, Fine Dining, Pub etc.)\n        *   **Cuisine Type:** (e.g., North Indian, Thai, Biryani etc.)\n        *   **Cuisine Type:** Location (e.g., neighborhood, area etc.).\n        *   **Cost for Two:** (price for a meal for two people, pick only the value)\n    3.  Use chain-of-thought reasoning to guide the conversation.  This means thinking step-by-step about what information you need and asking questions th

In [28]:
# Getting the response from the LLM Assistant by passing the conversation to the Chat Completions API
debug_response_assistant = get_chat_completions(debug_conversation)
print(debug_response_assistant)

Great! It sounds like you have some specific preferences in mind. Let me update your profile based on what you've shared.

Here's what I have so far:

```json
{
  "Restaurant Type": "Fine Dining",
  "Cuisine Type": "North Indian",
  "Location": "Whitefield",
  "Cost for Two": 2000
}
```

Is this information correct? Would you like to add or change anything?


In [29]:
# Evaluates if the assistant has captured the user's profile clearly.
def intent_confirmation_layer(response_assistant):

    prompt = f"""
    You are a senior evaluator who has an eye for detail.
    The input text will contain a user requirement captured through keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    {{
      "Restaurant Type": null,
      "Cuisine Type": null,
      "Location": null,
      "Cost for Two": null
    }}
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    THought 3 - Think carefully before the answering.
    """

    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model=model,
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 123
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

In [30]:
debug_response_assistant

'Great! It sounds like you have some specific preferences in mind. Let me update your profile based on what you\'ve shared.\n\nHere\'s what I have so far:\n\n```json\n{\n  "Restaurant Type": "Fine Dining",\n  "Cuisine Type": "North Indian",\n  "Location": "Whitefield",\n  "Cost for Two": 2000\n}\n```\n\nIs this information correct? Would you like to add or change anything?'

In [31]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
display(debug_confirmation)

{'result': 'Yes'}

In [32]:
# Checks if the final understanding of the user's profile is returned by the assistant as a Python dictionary.
def dictionary_present(response):
    delimiter = "####"


    prompt = f"""
    You are a helpful assistant designed to extract specific information from user input. 
    Your task is to identify and extract a Python dictionary from the given text and return it in JSON format.

    The input string will contain a Python dictionary with the following keys and value constraints:

    *   **'Restaurant Type'**
    *   **'Cuisine Type'**
    *   **'Location'**
    *   **'Cost for Two'**: Value must be a numerical value (integer or float represented as a string).

    Your output should be a valid JSON string representing the extracted dictionary.  
    If the input string does not contain a dictionary matching this format, or if the values do not conform to the constraints, 
    return an empty JSON object: `{{}}`.  
    Do not include any surrounding text or explanations in your response, only the JSON string.
    """
    
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format=True)

    return confirmation

In [33]:
debug_response_assistant

'Great! It sounds like you have some specific preferences in mind. Let me update your profile based on what you\'ve shared.\n\nHere\'s what I have so far:\n\n```json\n{\n  "Restaurant Type": "Fine Dining",\n  "Cuisine Type": "North Indian",\n  "Location": "Whitefield",\n  "Cost for Two": 2000\n}\n```\n\nIs this information correct? Would you like to add or change anything?'

In [34]:
response_dict_n = dictionary_present(debug_response_assistant)
print(response_dict_n)

{'Restaurant Type': 'Fine Dining', 'Cuisine Type': 'North Indian', 'Location': 'Whitefield', 'Cost for Two': '2000'}


In [35]:
type(response_dict_n)

dict

In [95]:
# Compares user profile with restaurants dataset and comes back with the top 5 recommendations

def compare_restaurants_with_user(user_req_dict):
    # Creating a copy of the DataFrame 
    filtered_rests = rests_df.copy()
    #print(filtered_rests.shape)

    # Extracting the budget value from user_requirements and converting it to an integer
    user_budget = int(user_req_dict.get('Cost for Two'))
    #print(user_budget)
    # Filtering restaurants based on the budget
    filtered_rests = filtered_rests[filtered_rests['avg cost (two people)'] <= user_budget]
    #print(filtered_rests.shape)
    
    # Filtering restaurants based on the location
    user_location = user_req_dict.get('Location')
    #print(user_location)
    filtered_rests = filtered_rests[filtered_rests['area'] == user_location]
    #print(filtered_rests.shape)
    
    # Iterating over each restaurant in the filtered DataFrame to verify based on user requirements
    for index, row in filtered_rests.iterrows():
        #print(row)
        # Filtering restaurants based on the restaurant type
        user_req = user_req_dict.get('Restaurant Type')
        #print(user_req)
        cur_value = row['restaurant type']
        cur_value = cur_value.strip()
        cur_list = cur_value.split(',')
        clean_list = [s.strip() for s in cur_list]
        
        if(not user_req in clean_list):
            filtered_rests.drop(index, inplace=True)
            continue
        
        # Filtering restaurants based on the cuisine type
        user_req = user_req_dict.get('Cuisine Type')
        #print(user_req)
        cur_value = row['cuisines type']
        cur_value = cur_value.strip()
        cur_list = cur_value.split(',')
        clean_list = [s.strip() for s in cur_list]
        
        if(not user_req in clean_list):
            filtered_rests.drop(index, inplace=True)

    # Sorting restaurants by rating in descending order and selecting the top 5
    top_rests = filtered_rests.sort_values('rate (out of 5)', ascending=False).head(5)
    top_rests_json = top_rests.to_json(orient='records')  # Converting to JSON format

    # top restaurants
    return top_rests_json

In [77]:
test_1 = """Great! Thank you for sharing your preferences. Let me update your user profile dictionary with the information 
you provided.\n\n- **Restaurant Type:** Casual Dining\n- **Cuisine Type:** North Indian\n- **Location:** Whitefield\n- **Cost for Two:** 2000\n\nNow, just to confirm, is this all correct?"""

In [78]:
response_dict_1 = dictionary_present(test_1)
print(response_dict_1)

{'Restaurant Type': 'Casual Dining', 'Cuisine Type': 'North Indian', 'Location': 'Whitefield', 'Cost for Two': '2000'}


In [79]:
top_restaurants = compare_restaurants_with_user(response_dict_1)
top_restaurants

(6984, 10)
2000
(6876, 10)
Whitefield
(248, 10)


'[{"restaurant name":"Punjab Grill","restaurant type":"Casual Dining","rate (out of 5)":4.8,"num of ratings":650,"avg cost (two people)":2000.0,"online_order":"Yes","table booking":"No","cuisines type":"North Indian","area":"Whitefield","local address":"Whitefield"},{"restaurant name":"Salt - Indian Restaurant - Bar & Grill","restaurant type":"Casual Dining","rate (out of 5)":4.4,"num of ratings":2073,"avg cost (two people)":1500.0,"online_order":"No","table booking":"Yes","cuisines type":"North Indian","area":"Whitefield","local address":"Whitefield"},{"restaurant name":"The Creek - The Den Bengaluru","restaurant type":"Casual Dining","rate (out of 5)":4.4,"num of ratings":383,"avg cost (two people)":2000.0,"online_order":"No","table booking":"Yes","cuisines type":"North Indian, Continental, Chinese","area":"Whitefield","local address":"Whitefield"},{"restaurant name":"Nook - Aloft Bengaluru Cessna Business Park","restaurant type":"Casual Dining","rate (out of 5)":4.2,"num of ratings"

In [80]:
# Get output in JSON Format
top_rests_json = json.loads(top_restaurants)
top_rests_json

[{'restaurant name': 'Punjab Grill',
  'restaurant type': 'Casual Dining',
  'rate (out of 5)': 4.8,
  'num of ratings': 650,
  'avg cost (two people)': 2000.0,
  'online_order': 'Yes',
  'table booking': 'No',
  'cuisines type': 'North Indian',
  'area': 'Whitefield',
  'local address': 'Whitefield'},
 {'restaurant name': 'Salt - Indian Restaurant - Bar & Grill',
  'restaurant type': 'Casual Dining',
  'rate (out of 5)': 4.4,
  'num of ratings': 2073,
  'avg cost (two people)': 1500.0,
  'online_order': 'No',
  'table booking': 'Yes',
  'cuisines type': 'North Indian',
  'area': 'Whitefield',
  'local address': 'Whitefield'},
 {'restaurant name': 'The Creek - The Den Bengaluru',
  'restaurant type': 'Casual Dining',
  'rate (out of 5)': 4.4,
  'num of ratings': 383,
  'avg cost (two people)': 2000.0,
  'online_order': 'No',
  'table booking': 'Yes',
  'cuisines type': 'North Indian, Continental, Chinese',
  'area': 'Whitefield',
  'local address': 'Whitefield'},
 {'restaurant name': '

In [91]:
# Initializes the recommendations conversation
def initialize_conv_reco(products):
    system_message = f"""
    You are an restaurant recommendation expert and you are tasked with the objective to \
    solve the user queries about any restaurant from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each restaurant in the following format. Maintain the order in the user input:
    1. <Restaurant Name> : <Details like cuisines, location, cost, address etc.>, <Rating(out of 5)>
    2. <Restaurant Name> : <Details like cuisines, location, cost, address etc.>, <Rating (out of 5)>

    """
    user_message = f""" These are the user's input restaurants: {products}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]

    return conversation

In [92]:
debug_conversation_reco = initialize_conv_reco(top_restaurants)
debug_conversation_reco

[{'role': 'system',
  'content': '\n    You are an restaurant recommendation expert and you are tasked with the objective to     solve the user queries about any restaurant from the catalogue in the user message     You should keep the user profile in mind while answering the questions.\n    Start with a brief summary of each restaurant in the following format. Maintain the order in the user input:\n    1. <Restaurant Name> : <Details like cuisines, location, cost, address etc.>, <Rating(out of 5)>\n    2. <Restaurant Name> : <Details like cuisines, location, cost, address etc.>, <Rating (out of 5)>\n\n    '},
 {'role': 'user',
  'content': ' These are the user\'s input restaurants: [{"restaurant name":"Punjab Grill","restaurant type":"Casual Dining","rate (out of 5)":4.8,"num of ratings":650,"avg cost (two people)":2000.0,"online_order":"Yes","table booking":"No","cuisines type":"North Indian","area":"Whitefield","local address":"Whitefield"},{"restaurant name":"Salt - Indian Restaura

In [83]:
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [84]:
print(moderation_check("I want to kill Ravan."))
print(moderation_check("I need good food"))

Flagged
Not Flagged


In [85]:
# The final orchestration function that manages the whole conversation with the user 
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    display(introduction + '\n')

    top_restaurants = None

    user_input = ''

    while(user_input != "exit"):

        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_restaurants is None:

            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_completions(conversation)
            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break


            confirmation = intent_confirmation_layer(response_assistant)

            print("Intent Confirmation Yes/No:",confirmation.get('result'))

            if "No" in confirmation.get('result'):
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")

            else:
                print("\n" + str(response_assistant) + "\n")
                print('\n' + "Variables extracted!" + '\n')

                response = dictionary_present(response_assistant)

                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_restaurants = compare_restaurants_with_user(response)

                print("top restaurants recommended are", top_restaurants)

                conversation_reco = initialize_conv_reco(top_restaurants)

                conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response)})

                recommendation = get_chat_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    display("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "assistant", "content": str(recommendation)})

                print(str(recommendation) + '\n')
        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

In [88]:
dialogue_mgmt_system()

"Hello! I'm here to help you find the perfect restaurant. To start, can you tell me a bit about what kind of dining experience you're looking for? For example, do you prefer casual dining, fine dining, a pub, or something else?\n"

Looking for a fine dining restaurant in Whitefield
Intent Confirmation Yes/No: No

Great choice! Fine dining is always a wonderful experience. To tailor the recommendation to your taste, could you let me know what type of cuisine you prefer? For example, do you enjoy North Indian, Italian, Thai, or something else?

North Indian
Intent Confirmation Yes/No: No

Excellent! North Indian cuisine offers such a rich variety of flavors. Now, could you tell me what your budget is? Specifically, what is the cost for two people for a meal, roughly?

2000
Intent Confirmation Yes/No: Yes

Thank you for that information! So far, I have:

- **Restaurant Type:** Fine Dining
- **Cuisine Type:** North Indian
- **Location:** Whitefield
- **Cost for Two:** 2000

Before I finalize this user profile, is there anything else specific you'd like to add or clarify? For example, do you have a preference for any particular dishes or ingredients you love or dislike?


Variables extracted!

Thank you for providing 

In [94]:
dialogue_mgmt_system()

"Hello! I'm here to help you find the perfect restaurant. To start, can you tell me a bit about what kind of dining experience you're looking for? For example, do you prefer casual dining, fine dining, a pub, or something else?\n"

Looking for casual dining in HSR location
Intent Confirmation Yes/No: No

Great start! It sounds like you're interested in casual dining in the HSR area. Would you like to share what type of cuisine you're in the mood for? For example, do you prefer North Indian, Italian, Thai, or something else?

North Indian
Intent Confirmation Yes/No: No

Got it! You’re looking for casual dining options with North Indian cuisine in the HSR area. Now, could you please tell me your budget? Specifically, what is the cost for two people for a meal?

2000
Intent Confirmation Yes/No: Yes

Thanks for sharing that! So far, here's what we have for your profile:

- **Restaurant Type:** Casual Dining
- **Cuisine Type:** North Indian
- **Location:** HSR
- **Cost for Two:** 2000

Is there anything else you'd like to add or clarify? For example, do you have a preference for any specific dishes or any dietary restrictions?


Variables extracted!

Thank you for providing all the information. Kindly wait, while I fe